<a href="https://colab.research.google.com/github/blongho/machine-learning/blob/master/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1

This homework mainly evaluates if you are familar with NumPy to create machine learning related functions. Thus, all your functions here should be only made using NumPy, not any other libraries like pandas, scikit-learn, or scipy. We will run a complete machine learning process from preprocessing to model performance testing by using the functions we will develop.

### This assignment is made to practice the features of NumPy. Therefore, the use of a for loop is strictly prohibited. All problems can be solved without explicit loops. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In the lab session, we try to draw a regression line on the full dataset. Here in this homework, we will continue to use the same dataset we have used in the first lab, as well as the code we have made. However, this time we would like to create a model and even test it - by dividing the dataset into a training set and a test set. We will practice applying each function one after the other in the correct order. 

To practice NumPy efficiently, unlike the lab, we will only use the values of the dataset, ignoring Pandas properties such as column names.

- Run the block below to load the dataset (**X** and **y**).

In [ ]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=False)

X = california_housing.data
y = california_housing.target

Now, we are ready to use both features and labels. To handle it correctly, you need to be familiar with its axis concept as it no longer has indices and columns that you can check by printing the variable.

In [ ]:
X

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [ ]:
y

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

### 1. Preprocessing

The first task is to open the dataset and preprocess it into the form that the model can understand. It involves imputation, train_test_split, standardization, and normalization. Some functions are already covered by the first lab, so if you finished the lab before, you can freely bring your code here to finish your homework.



First, we would like to apply both standardization and normalization. You can re-use your lab functions here if you have finished your lab tasks.


- Standardization: Make features have the same standard deviaton and mean.

- Normalization: Make the range of value normalized into [0, 1].

In [ ]:
def standardize(data):
  """
  Input: NumPy ndarray
  Output: NumPy ndarray with column mean == 0 and std == 1
  """
  df = (data-data.mean(axis=0))/data.std(axis=0)
  return df

def normalize(data):
  """"
  Input: NumPy ndarray
  Output: NumPy ndarray with column min == 0 and max == 1
  """
  df = (data - data.min(axis=0))/(data.max(axis=0) - data.min(axis=0))
  return df

Let's apply both functions separately and create X_standardized and X_normalized.

In [ ]:
X_standardized = standardize(X)
X_normalized = normalize(X)

We may also need to check if those functions are correctly made. Create a function to check the dataset's min, max, mean, std of each feature. You can re-use your lab function (**describe**) but this time you are not allowed to use pandas dataframe. There is no expected format for this function if you are successfully able to plot four statistics (min, max, mean, std).

In [ ]:
def describe(data: np.array):
  """
  Describe four statistics of the dataset.
  
  Input: NumPy ndarray
  Output: vertical min, max, mean, standard deviation
  """
  min_values = data.min(axis=0)
  #np.min(data, axis=0)
  max_values = data.max(axis=0)
  avg_values = data.mean(axis=0)
  std_values = data.std(axis=0)
 
  #print(min_values, max_values, avg_values, std_values)
  print("Min\t", end="")

  for val in min_values:
    print(f"{round(val, 4):>8}", end="\t")

  print("\nMax\t", end="")
  for val in max_values:
    print(f"{round(val, 4):>8}", end="\t")
  
  print("\nMean\t", end="")
  for val in avg_values:
    print(f"{round(val, 4):>8}", end="\t")

  print("\nStd\t", end="")
  for val in std_values:
    print(f"{round(val, 4):>8}", end="\t")
  return None

Using this function, let's check if your **standardize** and **normalize** functions are correctly working. Your output should be the same as the one below.

In [ ]:
describe(X_standardized)

Min	 -1.7743	 -2.1962	 -1.8523	 -1.6108	 -1.2561	  -0.229	 -1.4476	  -2.386	
Max	  5.8583	  1.8562	 55.1632	 69.5717	 30.2503	119.4191	  2.9581	  2.6253	
Mean	     0.0	     0.0	     0.0	    -0.0	    -0.0	     0.0	    -0.0	    -0.0	
Std	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	

In [ ]:
describe(X_normalized)

Min	     0.0	     0.0	     0.0	     0.0	     0.0	     0.0	     0.0	     0.0	
Max	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	
Mean	  0.2325	   0.542	  0.0325	  0.0226	  0.0399	  0.0019	  0.3286	  0.4761	
Std	   0.131	  0.2468	  0.0175	   0.014	  0.0317	  0.0084	   0.227	  0.1996	

However, this is not a complete setting we would like to have, as we want to train the model and also test it. That means we need to divide the dataset into two parts: {Training set, Test set} and only use the training set to train the model. We also need to create the function for it.

In [ ]:
def train_test_split(X, y, test_ratio = 0.3):
  # simulation
  # cross-val
  
  """
  Input:
    - X: features
    - y: labels
    - test_ratio: ratio of the test set
    
  Output:
    - X_train: separated training instances
    - X_test: separated test instances
    - y_train: separated training labels
    - y_test: separated test labels
  
  1. Randomly shuffled indices of the size of the data instances
  2. Divide the indices into two parts with the ratio of [1-test ratio:test ratio]
  3. Select training instances and labels with the first set of indices and test instances and labels with the second set of indices
  4. Return the training set and the test set
  """
  shuffled_index_list = np.random.permutation(len(X))
  number_of_test_set = int(len(X)*test_ratio)
  test_set_indices = shuffled_index_list[:number_of_test_set]
  training_set_indices = shuffled_index_list[number_of_test_set:]
 
  X_train = X[training_set_indices]
  X_test = X[test_set_indices]
  y_train =y[training_set_indices]
  y_test = y[test_set_indices]
  
  return X_train, X_test, y_train, y_test

Split your dataset into training and test sets with `test ratio = 0.3`.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.3)

After applying your train_test_split function, you can check the shape of each subset. The training set should have 14,448 rows while the test set might have 6,192 records. Uncommend the below line.

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14448, 8), (6192, 8), (14448,), (6192,))

You may remember, when you apply standardization or normalization on both training and test sets, you should not use any statistics from the test set. This means that you should use mean and standard deviation (or max and min values) of the training set and use those statistics to avoid cheating and make a valid model. 

- Create two functions (**apply_standardization**, **apply_normalization**) that uses training set's statistics and apply standardization or normalization to both sets.

In [ ]:
def apply_standardization(X_train, X_test):
  """
  Input:
    - X_train: training instances
    - X_test: test instances

  Output:
    - X_train_standardized
    - X_test_standardized

  Use training set's mean and standard deviation to standardize both training and test sets
  """
  x_train = (X_train-X_train.mean(axis=0))/X_train.std(axis=0)
  x_test = (X_test - X_train.mean(axis=0))/X_train.std(axis=0)
  return x_train, x_test

In [ ]:
def apply_normalization(X_train, X_test):
  """
  Input:
    - X_train
    - X_test

  Output:
    - X_train_standardized
    - X_test_standardized
  """
  x_train = (X_train - X_train.min(axis=0))/(X_train.max(axis=0) - X_train.min(axis=0))
  y_test = (X_test - X_train.min(axis=0))/(X_test.max(axis=0) - X_train.min(axis=0))
  return x_train, y_test

- Apply two functions (**apply_standardization**, **apply_normalization**) to created standardized and normalized datasets.

In [ ]:
X_train_standardized, X_test_standardized = apply_standardization(X_train, X_test)
X_train_normalized, X_test_normalized = apply_normalization(X_train, X_test)

Check the statistics using describe method. Test set should NOT have zero mean and standard deviation 1 or zero min and one max. Good test set however might show close value to zero or one.

In [ ]:
describe(X_train_standardized)

Min	 -1.7712	 -2.1863	 -1.9372	 -1.8625	 -1.2617	 -0.4498	  -1.447	 -2.3707	
Max	  5.8628	  1.8526	 57.7442	  60.006	   30.62	117.1182	   2.929	  2.5074	
Mean	    -0.0	    -0.0	     0.0	    -0.0	    -0.0	     0.0	     0.0	    -0.0	
Std	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	

In [ ]:
describe(X_test_standardized)

Min	 -1.7712	 -2.1071	 -1.8721	 -1.3786	 -1.2599	 -0.3719	 -1.4424	  -2.311	
Max	  5.8628	  1.8526	 53.7775	  80.619	 24.2614	243.4391	  2.8593	  2.6269	
Mean	  0.0116	  0.0087	  0.0056	  0.0132	  0.0313	  0.0567	 -0.0306	  0.0292	
Std	  1.0006	  0.9889	  1.1485	  1.4636	  1.0388	  3.3933	  0.9771	  0.9906	

In [ ]:
describe(X_train_normalized)

Min	     0.0	     0.0	     0.0	     0.0	     0.0	     0.0	     0.0	     0.0	
Max	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	
Mean	   0.232	  0.5413	  0.0325	  0.0301	  0.0396	  0.0038	  0.3307	   0.486	
Std	   0.131	  0.2476	  0.0168	  0.0162	  0.0314	  0.0085	  0.2285	   0.205	

In [ ]:
describe(X_test_normalized)

Min	     0.0	  0.0196	  0.0012	  0.0059	  0.0001	  0.0003	  0.0011	   0.012	
Max	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	     1.0	
Mean	  0.2335	  0.5435	  0.0349	  0.0227	  0.0507	  0.0021	  0.3289	  0.4802	
Std	  0.1311	  0.2448	  0.0206	  0.0177	  0.0407	  0.0139	  0.2269	  0.1982	

### 2. Linear regression

Now we are ready to train and test our model. We will continue to use the linear regression that we have made in the lab using the normal equation. 

- Create the **solver** function that creates a linear regression line and return the coefficents. You can re-use the function from the first lab.
- Here you should you all available features of the dataset.
- You should add one column representing a bias to your feature matrix.

The normal equation can be represented as follows:

$\theta = (\textbf{X}^T \cdot \textbf{X})^{-1} \cdot \textbf{X}^T \cdot \textbf{y}$

In [ ]:
def solver(X, y):
  """
  Get the weight and bias of the linear regression line on the dataset X, using the labels y.

  Input: 
   - X: features
   - y: labels
  Output:
   - theta: weight and bias of the linear regression
  """
  theta = np.linalg.inv(np.transpose(X)@X)@np.transpose(X)@y
  return theta

We will only run this solver function on the training set (**X_train**, **y_train**) to create the model and evalute it by using the test set.

- Run the **solve** function on **X_train** and **y_train** and save the result to **theta**.

In [ ]:
theta = solver(X_train_standardized, y_train)

We now have a complete model trained on the training set. Then the next interesting thing is to evaluate if the model is good enough by using the test set. To do this, we need to create a predict function that can return the expected value. 

- Create the **predict** function which put each instance into the regression equation to predict the value. DO NOT USE ANY LOOP.

In [ ]:
def predict(X, theta):
  """
  Input: 
   - X: data instances to predict
   - theta: trained regression coefficients

  Output:
   - y_hat: predicted values (X @ weight) + bias
  """
  y_hat =theta.dot(np.transpose(X)) + np.ones(len(X))
  return y_hat

This predict function should be able to return the predicted value of the housing price. Then now we might want to return the mean squared error of the whole model. There can be many different metrics but here we would like to measure rooted mean squared error. RMSE can be calculated as follows: 

$RMSE = \sqrt{\frac{1}{n}\sum_{t=1}^{n}(\bar{y}_t - y_t)^2} $

- Create a function **rooted_mean_squared_error** that calculates the RMSE value.

In [ ]:
def rooted_mean_squared_error(X, y, theta):
  """
  Input:
    - X_test: data instances to test
    - y_test: true values of the corresponding data instances (X_test)
    - theta: trained regression coefficients

  Output:
    - RMSE: RMSE score

  Use predict function to calculate our predicted values.
  """

  rmse = np.sqrt(np.sum(np.square(np.subtract(y,theta)))/len(X))
  return rmse

- Run RMSE function to test our model created by our solver function. Use predict function to calculate our predicted values. Report the RMSE value.

Even though the RMSE is generally the preferred performance measure for
regression tasks, in some contexts you may prefer to use another function. For
example, suppose that there are many outlier districts. In that case, you may
consider using the mean absolute error (MAE). It's direct translation of l1 and l2 norm. The higher the norm index, the more it focuses on large values and
neglects small ones. This is why the RMSE is more sensitive to
outliers than the MAE. But when outliers are exponentially rare (like
in a bell-shaped curve), the RMSE performs very well and is
generally preferred.

MAE can be calculated as follows:

$MAE = \frac{1}{n}\sum_{t=1}^{n}|\bar{y}_t - y_t|$

- Implement a function for MAE **mean_absolute_error**, which receives the same parameters *X*, *y*, and *theta*.

In [ ]:
def mean_absolute_error(X, y, theta):
  """
  Input:
    - X_test: data instances to test
    - y_test: true values of the corresponding data instances (X_test)
    - theta: trained regression coefficients

  Output:
    - MAE: MAE score

  Use predict function to calculate our predicted values.
  """
  mae_score = (1/len(X)) * (np.sum(abs(np.subtract(y,theta))))
  
  return mae_score

Train your regression model on the training set and evaluate your method with two different scores: RMSE and MAE. Print two scores here.

In [ ]:
rmse_score = rooted_mean_squared_error(X_test_standardized, y_test, theta) # CHANGE IT!
mae_score = mean_absolute_error(X_test_standardized, y_test, theta) # CHANGE IT!

ValueError: ignored

In [ ]:
rmse_score, mae_score

### 4. Linear regression with regularization

You have learned the Ridge regression in the lecture. Fortunately, the Ridge regression also can be represented as a closed form solution with the normal equation. 

Your task here is to create a variant of your previous solver function supporting the Ridge regression. 

A closed form solution to Ridge can be represented as follows:

$\theta = (\textbf{X}^T \cdot \textbf{X} + \alpha\textbf{I})^{-1} \cdot \textbf{X}^T \cdot \textbf{y}$

where $\textbf{I}$ is an $(n+1) \times (n+1) $ identity matrix, since the feature matrix also includes the bias column.


In [ ]:
def solver_with_ridge(X, y, alpha):
  """
  Get the weight and bias of the linear regression line on the dataset X, using the labels y.

  Input: 
   - X: features
   - y: labels
  Output:
   - theta: weight and bias of the ridge regression
  """
  I = (X+1)*(X+1)
  print(I)
  thetha = np.linalg.inv(np.transpose(X)@X + alpha*I)@np.transpose(X)@y

  return theta 

Here, compare the performances changing the $\alpha$ value. Use the $\alpha$ value from 0 to 30 in increments of 0.1. Use RMSE as a score metric. Save those 300 scores into the list `scores`. 

- To iterate different $\alpha$s, you can use a loop for your convenience.

In [ ]:
alpha = np.arange(0,30,0.1)
scores = []
for alph in range(len(alpha)):
  print("Current value of alpha is ",alpha)
  scores.append(solver_with_ridge(X_standardized, y, alph))

print(scores)

Plot the graph of different scores here. If you saved all scores in the list `scores`, you can simply run the block below. The resulting plot behaves in a different way based on your split training and test sets. Sometimes, the error just decreases or increases, but you can also see that the error decreases first, but after some point, it starts to increase. If you are interested, repeat many times to check different plots and you can even change the range from [0, 30] to something else. Uncomment the block below!

In [ ]:
plt.plot(np.arange(0, 30, 0.1), scores)
plt.xlabel("alpha")
plt.ylabel("RMSE")
plt.show()

### 5. Model validation

So far, you simply had one test set and one training set. Now the question is if those sets were enough to represent the dataset's distribution. To overcome this problem, various validation methods have been developed and used such as cross-validation or repeated holdout test. Here, you will develop one function that performs the repeated holdout test. The key strategy of it is to create a completely different training and test set pair for each iteration. You simply iterate holdout test that we performed many (k) times and return the average score.

* You are allowed to use a for loop to iterate k different tests. However, you are not allowed to use the loop to create different indices to divide the dataset.
* You can call the `train_test_split` function you have developed above if it helps your development process.

In [ ]:
def repeated_hold_out(X, y, k, test_ratio):
  """
  Input:
    - X: features
    - y: labels
    - test_ratio: ratio of the test set
  Output:
    - score: the average of k different test scores
  
  1. Iterate k times to perform k validation processes.
  2. For each iteration, split the dataset into the training and test sets with *random* indices.
   - Note that each iteration should create different training and test sets.
  3. Use *standardization* to fix the scale of the dataset, you should only use the training set's properties.
  4. Fit your model with *solver* (without ridge) on the training set.
  5. Save your *RMSE* score into the list *scores*
  6. After all the iterations, return the average of *scores*.

  """
  scores = []
  for idx in range(k):
    x_train, x_test, y_train, y_test = train_test_split(X,y,test_ratio)
    x_train_st, x_test_st = apply_standardization(x_train, x_test)
    theta = solver(x_train_st, y_train)
    rmse_value = rooted_mean_squared_error(x_train, y_train, theta)
    scores.append(rmse_value)
  return np.mean(scores)

Run the line below to return your holdout score.

In [ ]:
holdout_score = repeated_hold_out(X, y, k=5, test_ratio=0.2)
holdout_score

### 6. Put things together

It's time to put everything you have done together here. We will create a function that manages whole process from receiving raw datasets to returning performance metrics, by modifying the repeated holdout function. This will help you manage your process clearly since it must contain all the functions you use for your dataset (Later we will replace it with scikit-learn's pipeline technique for the same purpose) - By having these management functions, you can switch off some of the techniques or add more techniques in the middle without any problem.

* Complete three functions (preprocess, fit, run) following the instruction.

In [ ]:
def repeated_hold_out_2(X, y, k = 5, test_ratio = 0.2, norm_method = "standardization", eval_method = "RMSE", alpha = 0):
  """
  Input:
    - X: features
    - y: labels
    - test_ratio: ratio of the test set
  Output:
    - score: the average of k different test scores
  
  1. Iterate k times to perform k validation processes.
  2. For each iteration, split the dataset into the training and test sets with *random* indices.
   - Note that each iteration should create different training and test sets.
  3. Check the parameter *norm_method*
    - if norm_method == standardization:
      - Use *standardization* to fix the scale of the dataset, you should only use the training set's properties.
    - if norm_method == normalization:
      - Use *normalization* to fix the scale of the dataset, you should only use the training set's properties.
  4. Fit your model with *solver_with_ridge" on the training set. Use alpha from the parameter.
  5. Check the parameter "eval_method"
    - if eval_method == "RMSE"
      - Save your *RMSE* score into the list *scores*
    - if eval_method == "MAE"
      - Save your *MAE* score into the list *scores*

  6. After all the iterations, return the average of *scores*.

  """
  scores = []
  for idx in range(k):
    x_train, x_test, y_train, y_test = train_test_split(X,y,test_ratio)
    if norm_method == "standardization":
      x_train_st, y_train_st = apply_standardization(x_train, x_test)
    if norm_method == "normalization":
      x_train_st, y_train_st = apply_normalization(x_train, x_test)
    weight = solver_with_ridge(x_train_st, y_train_st, alpha)
    if eval_method == "RMSE":
      score = rooted_mean_squared_error(x_train_st, y_train_st, weight)
    if eval_method == "MAE":
      score = mean_absolute_error(x_train_st, y_train_st, weight)
    scores.append(score)

  return np.mean(scores)

Now you are ready to run various tasks by using this single function. Will the best model the same under RMSE or MAE? Will different k or test ratio result in different best model? You can do many different trials to find a good model.

- (Optional) Change a normalization method, an alpha value to find out the best classifier under either RMSE or MAE score. This task is completely optional and will not affect your homework grade.

In [ ]:
best_model = None

# END